In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [2]:
train_dataset = dsets.MNIST(root='./MNIST/', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./MNIST/', 
                           train=False, 
                           transform=transforms.ToTensor())

Processing...
Done!


In [ ]:
class ActNorm(nn.Module):
    def __init__(self, h, w, c):
        super().__init__()
        self.shape = (h, w, c)
        self.weights = nn.Parameter(torch.Tensor(c))
        self.bias = nn.Parameter(torch.Tensor(c))
        self.weights.data.uniform_(-0.1, 0.1)
        self.bias.data.uniform_(-0.1, 0.1)
        
    def forward(self, inp):
        return inp * self.weights + self.bias
    
    def log_determinant:
        return self.shape[0] * self.shape[1] * torch.log(torch.abs(self.weights))

In [ ]:
class InvertibleConv(nn.Module):
    def __init__(self, h, w, c):
        super().__init__()
        self.shape = (h, w, c)
        self.weights = nn.Parameter(torch.Tensor((c, c)))
        self.weights.data.uniform_(-0.1, 0.1) #надо нормально сделать
                                    
    def forward(self, inp):
        return torch.dot(inp, self.weights)
    
    def log_determinant:
        return self.shape[0] * self.shape[1] * torch.log(torch.det(self.weights))

In [ ]:
class GlowModel(nn.Module):
    def __init__(self):
        super().__init__()
        